In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
!pip install ipyplot pytorch_lightning timm
import pandas as pd
import ipyplot
import torch
import pytorch_lightning as pl
from sklearn.metrics import f1_score
%matplotlib inline

In [ ]:
import sys
sys.path.append('/content/src')

In [ ]:
from src.model import ImageClassifier
from src.utils import ImagesDataset, evaluate_model
from src.transforms import transform
from src import params

# Load dataset

In [ ]:
train_df = pd.read_csv('dataset/classification/train_labelled.csv')
train_df.head()

,img_path,label
0,dataset/classification/train_labelled/0.png,0
1,dataset/classification/train_labelled/1.png,0
2,dataset/classification/train_labelled/2.png,0
3,dataset/classification/train_labelled/3.png,0
4,dataset/classification/train_labelled/4.png,0


In [ ]:
toloka_df = pd.read_csv('dataset/classification/toloka_labelled.csv')
toloka_df.head()

,img_path,label
0,dataset/classification/train_unlabelled/0.png,0
1,dataset/classification/train_unlabelled/1.png,0
2,dataset/classification/train_unlabelled/10.png,0
3,dataset/classification/train_unlabelled/100.png,0
4,dataset/classification/train_unlabelled/1000.png,0


In [ ]:
# закинем результаты в общий датафрейм
train_df = train_df.append(toloka_df, ignore_index=True)

In [ ]:
train_df.sample(5)

,img_path,label
1470,dataset/classification/train_unlabelled/2140.png,0
2122,dataset/classification/train_unlabelled/2728.png,0
944,dataset/classification/train_unlabelled/1668.png,0
797,dataset/classification/train_unlabelled/1535.png,0
402,dataset/classification/train_unlabelled/118.png,0


In [ ]:
train_df.shape

(2200, 2)

In [ ]:
train_image_paths = train_df.img_path.values
train_labels = train_df.label.values

In [ ]:
len(train_image_paths)

2200

In [ ]:
ipyplot.plot_class_tabs(train_image_paths, train_labels)

# Prepare model

In [ ]:
train_dataset = ImagesDataset(train_image_paths, train_labels, transform=transform)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=params.BATCH_SIZE,
                                           num_workers=16,
                                           shuffle=False)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# Run macos_cert_fix.py if you are getting ssl errors
model = ImageClassifier(learning_rate=params.LEARNING_RATE)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/mobilenetv2_100_ra-b33bc2c4.pth" to /root/.cache/torch/hub/checkpoints/mobilenetv2_100_ra-b33bc2c4.pth


In [ ]:
trainer = pl.Trainer(
    max_epochs=params.MAX_EPOCHS,
    log_every_n_steps=-1, # Change to enable tensorboard logging
    gpus=-1, # Uncomment to use GPU training
)
trainer.fit(model, train_loader)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=-1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=-1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/configuration_validator.py:110: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  category=PossibleUserWarning,
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pyt

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


# Compute training set accuracy

In [ ]:
labels, predictions = evaluate_model(model, train_dataset)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/69 [00:00<?, ?it/s]

In [ ]:
f1_score(labels, predictions, labels=1, average='binary')

0.8514412416851441

# Save model

In [ ]:
trainer.save_checkpoint("model.ckpt")